In [1]:
import osmnx as ox
import networkx as nx
import pandas as pd
import os
from geopy.distance import geodesic

In [3]:
# --- 1. Load the Road Network Graph (G) and Aligned EV Stations (df_stations) ---
# These are loaded at the beginning of the script to ensure they are available.
# If continuing from the previous cell, these variables should already be in memory.
data_folder = ''

graph_file_path = os.path.join(data_folder, "road_network_Delhi_India.graphml")
G = ox.load_graphml(filepath=graph_file_path)
print(f"Loaded road network graph (Nodes: {len(G.nodes)}, Edges: {len(G.edges)})")

aligned_ev_file_path = os.path.join(data_folder, 'aligned_ev_stations.csv')
df_stations = pd.read_csv(aligned_ev_file_path)
print(f"Loaded aligned EV stations (Shape: {df_stations.shape})")

# Ensure 'nearest_node' is int type in case it wasn't saved/loaded as such
df_stations['nearest_node'] = df_stations['nearest_node'].astype(int)

# --- 2. Define Core Routing Functions ---

def get_nearest_node_to_point(graph, lat, lon):
    """
    Finds the nearest graph node to a given (latitude, longitude) point.
    Returns the node ID.
    """
    # ox.distance.nearest_nodes expects (graph, x(longitude), y(latitude))
    return ox.distance.nearest_nodes(graph, lon, lat)

def calculate_shortest_path(graph, origin_node, destination_node):
    """
    Calculates the shortest path between two nodes in the graph based on edge length.
    Returns a list of node IDs forming the path.
    """
    # Use Dijkstra's algorithm to find the shortest path based on 'length' attribute
    # 'length' is automatically calculated by osmnx for road segments
    return nx.shortest_path(graph, origin_node, destination_node, weight='length')

def find_charging_stations_along_route(route_nodes, df_ev_stations, buffer_meters=500):
    """
    Identifies charging stations that are along or very near the calculated route.
    """
    # Get coordinates of all nodes in the calculated route
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route_nodes]

    # Convert route_nodes list to a set for faster lookup
    route_nodes_set = set(route_nodes)

    nearby_stations = []
    for index, station in df_ev_stations.iterrows():
        station_coords = (station['latitude'], station['longitude'])

        # Option A: Check if station's nearest_node is directly ON the route
        if station['nearest_node'] in route_nodes_set:
            nearby_stations.append(station)
        # Option B: Check if station is within a certain buffer distance from ANY route node
        # This is more robust but computationally heavier if buffer_meters is large or route is long
        # For hackathon, Option A is simpler. If A doesn't yield enough, consider B.
        # (We will stick with Option A for now for simplicity and performance)

        # Let's add a simplified check for geographical proximity to any route coordinate
        # (This is a more relaxed check for stations "near" the route, not just on a node)
        # You could optimize this with a k-d tree or spatial index for very large routes/station lists.
        # For now, we'll iterate through all route coords for each station.

        # We can simplify this. If the station's `nearest_node` is *not* in the exact path,
        # but is very close geographically to *any* node in the path, we might want to include it.
        # This requires iterating through route_coords and checking geodesic distance.
        # For a simpler approach, let's just use the `nearest_node` being directly in the path for now.
        # If the simple `in route_nodes_set` isn't enough, we can revisit.
        # Let's add a refined buffer check using nearest_nodes of stations directly against route nodes.

        # Refined approach: Find all charging stations within a 500m radius of ANY node on the route
        # This is computationally expensive, so use sparingly or optimize for large graphs/data
        # For hackathon, consider only stations whose 'nearest_node' is directly on the path for simplicity.
        # If the dataset for Delhi has very few stations, we might need to broaden this.
        # Given the data, let's stick to the simplest, most performant approach first: stations whose nearest node is *exactly* in the route path.

        # Let's refine for actual use: find stations whose geographic coordinates are near ANY node on the route.
        # This is better than just matching node IDs, as roads bend between nodes.
        # Instead of iterating through all route_coords for each station, let's filter stations
        # that are near the path.
        pass # We'll build the logic below after the first simple check.

    # Let's refine 'find_charging_stations_along_route' by filtering stations whose nearest node is ON the path
    # And then for those, check their actual geographic distance to the path.
    # This will be more robust.

    # Filter stations whose 'nearest_node' is one of the nodes in the calculated route
    # This is a simple, efficient first pass.
    stations_on_path_nodes = df_ev_stations[df_ev_stations['nearest_node'].isin(route_nodes_set)].copy()

    # Add distance from start of route as a sorting metric (optional but useful)
    # This calculates cumulative length along the route for each node
    route_distances = {}
    path_length = 0
    for i in range(len(route_nodes) - 1):
        u = route_nodes[i]
        v = route_nodes[i+1]
        edge_data = G.get_edge_data(u, v)
        if edge_data:
            # If there are multiple edges between u and v, pick one (e.g., the first)
            edge_length = edge_data[0]['length'] if isinstance(edge_data, dict) else edge_data['length']
            path_length += edge_length
        route_distances[v] = path_length # Distance to node v from origin
    route_distances[route_nodes[0]] = 0 # Origin node is at distance 0

    # Calculate distance of each station on the path from the route origin
    stations_on_path_nodes['distance_from_route_origin'] = stations_on_path_nodes['nearest_node'].map(route_distances)
    stations_on_path_nodes.sort_values(by='distance_from_route_origin', inplace=True)

    return stations_on_path_nodes
# --- End of function definitions ---

# --- 3. Example Usage: Plan a Trip ---
print("\n--- Planning Example Trip (Delhi to Noida Area) ---")

# Define Origin and Destination - Pick two points in Delhi/NCR from your data or manually.
# We need to manually pick Lat/Lon for now. Let's use two example points.
# Example 1: EESL R block GK1 (Delhi)
origin_lat, origin_lon = 28.547217, 77.244324
# Example 2: EESL Tata Advance Systems Noida (Noida)
destination_lat, destination_lon = 28.608947, 77.362725

print(f"Origin: ({origin_lat}, {origin_lon})")
print(f"Destination: ({destination_lat}, {destination_lon})")

# Find nearest graph nodes for origin and destination
origin_node = get_nearest_node_to_point(G, origin_lat, origin_lon)
destination_node = get_nearest_node_to_point(G, destination_lat, destination_lon)

print(f"Nearest origin node: {origin_node}")
print(f"Nearest destination node: {destination_node}")

# Calculate shortest path
if origin_node and destination_node:
    try:
        route = calculate_shortest_path(G, origin_node, destination_node)
        route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length')) / 1000 # in km
        print(f"Calculated shortest path with {len(route)} nodes.")
        print(f"Estimated route length: {route_length:.2f} km.")

        # Find charging stations along this route
        # This is where the magic happens for charging stops
        charging_stops = find_charging_stations_along_route(route, df_stations)

        print(f"\n--- Found {len(charging_stops)} potential charging stops along the route: ---")
        if not charging_stops.empty:
            display(charging_stops[['station_name', 'address', 'city', 'latitude', 'longitude', 'distance_from_route_origin']].head(10)) # Display first 10
        else:
            print("No charging stations found directly on the calculated route segment.")

        # --- 4. Optional: Visualize the Route ---
        # This will pop up a map or show inline in Jupyter.
        print("\n--- Visualizing the calculated route (might pop up a window or display inline) ---")
        fig, ax = ox.plot_graph_route(G, route, node_size=0, bgcolor='k', show=False, close=True, route_color='cyan', route_linewidth=4)

        # Plot charging stations on the map
        # Filter stations that are directly on the route (or their nearest node is on the route)
        stations_on_route_coords = charging_stops[['latitude', 'longitude']].values.tolist()
        if stations_on_route_coords:
            ax.scatter([lon for lat, lon in stations_on_route_coords],
                       [lat for lat, lon in stations_on_route_coords],
                       color='red', s=50, edgecolors='white', zorder=5, label='Charging Stops')
            ax.legend() # Show legend if charging stops are plotted

        plt.show() # Display the plot

    except nx.NetworkXNoPath:
        print("Error: No path found between the origin and destination nodes. They might be disconnected in the graph or too far.")
    except Exception as e:
        print(f"An error occurred during path calculation or visualization: {e}")
else:
    print("Could not find nearest nodes for both origin and destination. Check input coordinates.")

print("\n--- Core Routing Algorithm step complete. ---")

Loaded road network graph (Nodes: 183054, Edges: 498361)
Loaded aligned EV stations (Shape: (116, 12))

--- Planning Example Trip (Delhi to Noida Area) ---
Origin: (28.547217, 77.244324)
Destination: (28.608947, 77.362725)
Nearest origin node: 6436645068
Nearest destination node: 1820933814
Calculated shortest path with 118 nodes.
Estimated route length: 15.51 km.

--- Found 2 potential charging stops along the route: ---


,station_name,address,city,latitude,longitude,distance_from_route_origin
1,EESL R block GK1,"SDMC Parking, R Block, GK-1, DELHI-110016",Delhi,28.547217,77.244324,0.000000
3,EESL Tata Advance Systems Noida,"Near Tata Advance Systems, sector- 59, Noida",Noida,28.608947,77.362725,15511.000425



--- Visualizing the calculated route (might pop up a window or display inline) ---
An error occurred during path calculation or visualization: name 'plt' is not defined

--- Core Routing Algorithm step complete. ---
